In [ ]:
import numpy as np
np.random.seed(0)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib.ticker import NullFormatter
%matplotlib inline
import seaborn as sns
sns.set(style="ticks")
from mpl_toolkits.mplot3d import Axes3D
sns.set(palette="bright",style="ticks")

import pandas as pd

# Settings

In [ ]:
alpha = 0.05

mnist_ff = f"{os.getcwd()}/experiments/robust_cv/qmnist/"
cifar_ff = f"{os.getcwd()}/experiments/robust_cv/cifar-10/"
imagenet_ff = f"{os.getcwd()}/experiments/robust_cv/imagenet/"
out_ff = f"{os.getcwd()}/experiments/robust_cv/figures/"

# Run models

In [ ]:
!python cifar10_mnist_experiment.py --nTrials 20 --alpha_coverage 0.05 --dataset "QMNIST" --epochs_quantile 10000 --delta_slab 0.05 --alpha_slab 0.05 --n_slabs_directions 1000 --rho_validation "slab_quantile" --experiment_name "slab_quantile-3"

!python cifar10_mnist_experiment.py --nTrials 20 --alpha_coverage 0.05 --dataset "QMNIST" --epochs_quantile 10000 --delta_slab 0.05 --alpha_slab 0.05 --n_slabs_directions 1000 --rho_validation "learnable_direction_OLS" --experiment_name "slab_quantile-1"

!python cifar10_mnist_experiment.py --nTrials 20 --alpha_coverage 0.05 --dataset "QMNIST" --epochs_quantile 10000 --delta_slab 0.05 --alpha_slab 0.05 --n_slabs_directions 1000 --rho_validation "learnable_direction_SVM" --experiment_name "slab_quantile-2"

In [ ]:
!python cifar10_imagenet_experiment.py --nTrials 20 --alpha_coverage 0.1 --dataset "imagenet" --cqc_epochs_quantile 500 --delta_slab 0.1 --alpha_slab 0.1 --n_slabs_directions 1000 --rho_validation "slab_quantile" --experiment_name "slab_quantile-3"

!python cifar10_imagenet_experiment.py --nTrials 20 --alpha_coverage 0.1 --dataset "imagenet" --cqc_epochs_quantile 500 --delta_slab 0.1 --alpha_slab 0.1 --n_slabs_directions 1000 --rho_validation "learnable_direction_OLS" --experiment_name "slab_quantile-1"

!python cifar10_imagenet_experiment.py --nTrials 20 --alpha_coverage 0.1 --dataset "imagenet" --cqc_epochs_quantile 500 --delta_slab 0.1 --alpha_slab 0.1 --n_slabs_directions 1000 --rho_validation "learnable_direction_SVM" --experiment_name "slab_quantile-2"

In [ ]:
!python cifar10_imagenet_experiment.py --nTrials 20 --alpha_coverage 0.05 --dataset "cifar-10" --cqc_epochs_quantile 10000 --delta_slab 0.1 --alpha_slab 0.1 --n_slabs_directions 1000 --rho_validation "slab_quantile" --experiment_name "slab_quantile-3"

!python cifar10_imagenet_experiment.py --nTrials 20 --alpha_coverage 0.05 --dataset "cifar-10" --cqc_epochs_quantile 10000 --delta_slab 0.1 --alpha_slab 0.1 --n_slabs_directions 1000 --rho_validation "learnable_direction_OLS" --experiment_name "slab_quantile-1"

!python cifar10_imagenet_experiment.py --nTrials 20 --alpha_coverage 0.05 --dataset "cifar-10" --cqc_epochs_quantile 10000 --delta_slab 0.1 --alpha_slab 0.1 --n_slabs_directions 1000 --rho_validation "learnable_direction_SVM" --experiment_name "slab_quantile-2"

# Make plots for mnist, cifar-10, image net

In [ ]:
def get_cvgs_and_lens(results):
    
    naive = results[results["Robustness"] == "NaiveConformal"]
    naive = naive.iloc[:,2:4]
    naive = naive.to_numpy()
    naive_cvgs = naive[:,1]
    naive_lens = naive[:,0]
    
    rob = results[results["Robustness"] == "RobustConformal"]
    rob = rob.iloc[:,2:4]
    rob = rob.to_numpy()
    rob_cvgs = rob[:,1]
    rob_lens = rob[:,0]
    
    return (naive_cvgs, naive_lens), (rob_cvgs, rob_lens)

In [ ]:
def make_plots(naive, rob, mode, out_ff, dataset, alpha):
    
    fontsize = 16
    
    obj_to_plot = np.vstack([naive, rob]).T
    
    f,a = plt.subplots(figsize=(8,5))
    a.boxplot(obj_to_plot,
              labels=["Standard", "Chi-squared"],
              showmeans=True,
              showfliers=True)

    a.tick_params(axis='both', labelsize=fontsize)
    a.set_title(mode, fontsize=fontsize+2)
    
    if mode == "Coverage":
        a.axhline((1-alpha), c='r', linestyle="-", linewidth=1)
        a.set_ylim([0.85,1])
    
    out_fp = (out_ff + dataset + "_" + mode + "s_boxplot.pdf").lower()
    f.savefig(out_fp, bbox_inches="tight")
    print("Saving figure to " + out_fp + ".")

In [ ]:
def make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                     svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                     slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                     out_ff, mode, dataset, alpha):
    
    fontsize = 16
    
    if mode == "Coverage":
        obj_to_plot = np.vstack([ols_naive_cvgs,
                                 slab_rob_cvgs,
                                 ols_rob_cvgs,
                                 svm_rob_cvgs]).T

    else:
        obj_to_plot = np.vstack([ols_naive_lens,
                                 slab_rob_lens,
                                 ols_rob_lens,
                                 svm_rob_lens]).T
        
    f,a = plt.subplots(figsize=(12,5))
    a.boxplot(obj_to_plot,
              labels=["Standard",
                      "Chi-squared, sampling",
                      "Chi-squared, regression",
                      "Chi-squared, classification"],
              showmeans=True,
              showfliers=True)

    a.tick_params(axis='both', labelsize=fontsize)
    a.tick_params(axis="x", rotation=45)
    a.set_title(mode, fontsize=fontsize+2)
    
    if mode == "Coverage":
        a.axhline((1-alpha), c='r', linestyle="-", linewidth=1)
        a.set_ylim([a.get_ylim()[0],1])
    
    out_fp = (out_ff + dataset + "_" + mode + "s_boxplot.pdf").lower()
    f.savefig(out_fp, bbox_inches="tight")
    print("Saving figure to " + out_fp + ".")

In [ ]:
def process_fp(fp_in, out_ff, dataset, alpha):
    
    results = pd.read_csv(fp_in).iloc[:,1:]
    results = results[results["Conformalization"] == "Marginal"]
    ((naive_cvgs, naive_lens), (rob_cvgs, rob_lens)) = get_cvgs_and_lens(results)
    
    make_plots(naive_cvgs, rob_cvgs, "Coverage", out_ff, dataset, alpha)
    make_plots(naive_lens, rob_lens, "Size", out_ff, dataset, alpha)
    
    return naive_cvgs, naive_lens, rob_cvgs, rob_lens

## MNIST

In [ ]:
ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens = process_fp(
    mnist_ff + "slab_quantile-1-summary.csv", out_ff, "mnist_ols", alpha)

svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens = process_fp(
    mnist_ff + "slab_quantile-2-summary.csv", out_ff, "mnist_svm", alpha)

slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens = process_fp(
    mnist_ff + "slab_quantile-3-summary.csv", out_ff, "mnist_slab_quantile", alpha)

make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                 svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                 slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                 out_ff, "Coverage", "mnist", alpha)

make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                 svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                 slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                 out_ff, "Size", "mnist", alpha)

## CIFAR

In [ ]:
ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens = process_fp(
    cifar_ff + "slab_quantile-1-summary.csv", out_ff, "cifar-10_ols", alpha)

svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens = process_fp(
    cifar_ff + "slab_quantile-2-summary.csv", out_ff, "cifar-10_svm", alpha)

slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens = process_fp(
    cifar_ff + "slab_quantile-3-summary.csv", out_ff, "cifar-10_slab_quantile", alpha)

make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                 svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                 slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                 out_ff, "Coverage", "cifar-10", alpha)

make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                 svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                 slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                 out_ff, "Size", "cifar-10", alpha)

## ImageNet

In [ ]:
alpha = 0.1

ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens = process_fp(
    imagenet_ff + "slab_quantile-1-summary.csv", out_ff, "imagenet_ols", alpha)

svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens = process_fp(
    imagenet_ff + "slab_quantile-2-summary.csv", out_ff, "imagenet_svm", alpha)

slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens = process_fp(
    imagenet_ff + "slab_quantile-3-summary.csv", out_ff, "imagenet_slab_quantile", alpha)

make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                 svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                 slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                 out_ff, "Coverage", "imagenet", alpha)

make_joint_plots(ols_naive_cvgs, ols_naive_lens, ols_rob_cvgs, ols_rob_lens,
                 svm_naive_cvgs, svm_naive_lens, svm_rob_cvgs, svm_rob_lens,
                 slab_naive_cvgs, slab_naive_lens, slab_rob_cvgs, slab_rob_lens,
                 out_ff, "Size", "imagenet", alpha)